In [1]:
!pip install tokenizers
import torch
!pip install transformers
from transformers import AutoTokenizer, AutoModel
import requests
import numpy as np
import tensorflow as tf
!pip install cohere
import cohere
!pip install mtranslate
from mtranslate import translate
import nltk
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 25.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 13.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
# Define the URLs for the Cohere API
co = cohere.Client('UmLRVaXvGMiR9FbQxBGcy42Pp78e1GqD84uzEdTX')
# Load the tokenizer and model
from transformers import *
# Load the BERT model
model_name = "bert-base-multilingual-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_flax_utils.py:24: FutureWarning: Importing `FlaxGenerationMixin` from `src/transformers/generation_flax_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import FlaxGenerationMixin` instead.
  warnings.warn(


loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/tokenizer_config.json


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.28.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size":

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/pytorch_model.bin
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model f

In [3]:
# Define a function to get the Cohere embedding for a sentence
def get_embedding(txt):
  embedding = co.embed(texts=list(txt),
                  model='embed-multilingual-v2.0')
  return np.array(embedding.embeddings[0])
  
# Define a function to translate a sentence from English to Yoruba
def translate_to_yoruba(en_sentence):
    # Get the Cohere embedding for the English sentence
    en_embedding = get_embedding(en_sentence)

    # Tokenize the input sentence
    inputs = tokenizer.encode_plus(en_sentence, add_special_tokens=True, return_tensors='pt')

    # Get the model output
    outputs = model(**inputs)

    # Get the last hidden state of the model as the sentence embedding
    bert_embedding = outputs.last_hidden_state.mean(dim=1).squeeze().detach().numpy()

    # Compute the cosine similarity between the Cohere embedding and the BERT embedding
    cosine_sim = np.dot(en_embedding, bert_embedding) / (np.linalg.norm(en_embedding) * np.linalg.norm(bert_embedding))
    print(cosine_sim)
    # If the cosine similarity is above a certain threshold, use the BERT model to translate the sentence
    if cosine_sim > 0.8:
        # Convert the model output to a Yoruba sentence
        yo_sentence = translate(tokenizer.decode(inputs['input_ids'][0][1:-1]), "yo")
    else:
        # If the cosine similarity is below the threshold, return the input sentence
        yo_sentence = en_sentence

    return yo_sentence



In [4]:
articles = ['Hello from Cohere!', 'مرحبًا من كوهير!', 'Hallo von Cohere!',  
   'Bonjour de Cohere!', '¡Hola desde Cohere!', 'Olá do Cohere!',  
   'Ciao da Cohere!', '您好，来自 Cohere！', 'कोहेरे से नमस्ते!','Die britische Regierung hat angekündigt, einen neuen digitalen Steuerdienst zu schaffen, der 2017 eingeführt werden soll. Der neue Dienst soll es den Menschen erleichtern, ihre Steuern zu zahlen, und wird online und über mobile Geräte verfügbar sein. Der neue Service ermöglicht es den Menschen auch, auf ihre Steuerunterlagen zuzugreifen und Änderungen an ihren Steuerzahlungen vorzunehmen.','The UK government has announced that it will be creating a new digital tax service that will be launched in 2017. The new service will be designed to make it easier for people to pay their taxes and will be available online and via mobile devices. The new service will also allow people to access their tax records and make changes to their tax payments.',
 "The housing deficit in Nigeria is estimated to be 17 million units, with the population expected to reach 200 million by 2050. The country's housing deficit is currently estimated at $22bn. The Federal Government has made several attempts to address the issue, including the creation of the Federal Mortgage Bank of Nigeria, the National Housing Fund and the Nigeria Mortgage Refinance Company. However, the housing sector is still in need of a major boost. The country's housing deficit is currently estimated at $22bn. The Federal Government has made several attempts to address the issue, including the creation of the Federal Mortgage Bank of Nigeria, the National Housing Fund and the Nigeria Mortgage Refinance Company. However, the housing sector is still in need of a major boost.",
 'The Federal Capital Territory Administration (FCTA) has warned residents of the capital city of Abuja against the proliferation of illegal housing estates in the city. The FCTA has also warned against the sale of land to unsuspecting Nigerians by land grabbers. The Director of Development Control in the FCTA, Mukhtar Galadima, gave the warning at a press briefing in Abuja. He said the FCTA had identified illegal housing estates located along the Iddo train station, Apo, Tapi Lugbe areas within the Abuja Municipal Area Council. He said the FCTA would demolish the illegal structures.']


for article in articles:
    yo_sentence = translate_to_yoruba(article)
    print('**********   Start   **********')
    print("English sentence:", article)
    print('*****************************')
    print("Yoruba sentence:", yo_sentence)
    print('**********   End   **********')

-0.054648134732720566
**********   Start   **********
English sentence: Hello from Cohere!
*****************************
Yoruba sentence: Hello from Cohere!
**********   End   **********
-0.052513795142159814
**********   Start   **********
English sentence: مرحبًا من كوهير!
*****************************
Yoruba sentence: مرحبًا من كوهير!
**********   End   **********
-0.005885364194045959
**********   Start   **********
English sentence: Hallo von Cohere!
*****************************
Yoruba sentence: Hallo von Cohere!
**********   End   **********
-0.027261036252387568
**********   Start   **********
English sentence: Bonjour de Cohere!
*****************************
Yoruba sentence: Bonjour de Cohere!
**********   End   **********
-0.0384668682674902
**********   Start   **********
English sentence: ¡Hola desde Cohere!
*****************************
Yoruba sentence: ¡Hola desde Cohere!
**********   End   **********
-0.018690834282978137
**********   Start   **********
English sentence: